In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.sql import Window

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

21/09/21 13:31:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


----

## Adult Data Set

https://archive.ics.uci.edu/ml/datasets/adult

In [2]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num","marital-status", "occupation", 
             "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", 
             "native-country", "earnings"]

df = spark.read.csv("adult.data", header=False, inferSchema=True, ignoreLeadingWhiteSpace=True)

df = df.select(*[f.col(old).alias(new) for old, new in zip(df.columns, col_names)]).drop("fnlwgt").dropna("any")
#df = df.toDF(*col_names).drop("fnlwgt").dropna("any")

df.show(3, vertical=True)

-RECORD 0----------------------------
 age            | 39                 
 workclass      | State-gov          
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Never-married      
 occupation     | Adm-clerical       
 relationship   | Not-in-family      
 race           | White              
 sex            | Male               
 capital-gain   | 2174               
 capital-loss   | 0                  
 hours-per-week | 40                 
 native-country | United-States      
 earnings       | <=50K              
-RECORD 1----------------------------
 age            | 50                 
 workclass      | Self-emp-not-inc   
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Married-civ-spouse 
 occupation     | Exec-managerial    
 relationship   | Husband            
 race           | White              
 sex            | Male               
 capital-gain   | 0                  
 capital-los

> **ZADANIA**:
1. Oblicz średnią `capital-gain` oraz `capital-loss` dla każdej narodowości - wynik uporządkuj malejąco po średnim zysku
2. Oblicz jaki procent kobiet zarabia ponad 50K w podziale na wykształcenie - wynik zaokrąglij do 2 miejsc po przecinku
3. Oblicz ile godzin średnio w ciągu tygodnia pracują osoby `Never-married` w stosunku do reszty społeczeństwa
4. W jakim wieku najczęściej zarabia sie powyżej 50K?
5. W podziale ze względu na zmienną `race` oblicz stosunek liczby kobiet do mężczyzn
6. Oblicz różnicę pomiędzy średnim czasem pracy osób uzyskujących `capital-gain` a resztą populacji
7. Oblicz róznicę wieku do średniego wieku osób z danym poziomem wykształcenia

In [3]:
df.createOrReplaceTempView("df")

In [4]:
# 1 Oblicz średnią capital-gain oraz capital-loss dla każdej narodowości - wynik uporządkuj malejąco po średnim zysku
df.groupBy("native-country").agg(f.avg("capital-gain").alias("avg-gain"), f.avg("capital-loss").alias("avg-loss"))\
.orderBy(f.desc(f.col("avg-gain"))).show()

+------------------+------------------+------------------+
|    native-country|          avg-gain|          avg-loss|
+------------------+------------------+------------------+
|             India|           3604.92|            131.12|
|             Japan|2546.2258064516127| 57.53225806451613|
|            Taiwan|2147.0392156862745| 154.8235294117647|
|              Iran|2044.6511627906978|126.81395348837209|
|             South|         1869.7375|           169.425|
|                 ?|1806.5180102915951|118.26586620926244|
|            Canada|1504.1322314049587|129.93388429752065|
|Dominican-Republic| 1476.057142857143| 25.17142857142857|
|            Greece| 1283.896551724138|295.44827586206895|
|     United-States|1089.2299280082277| 88.51059307507714|
|          Cambodia| 1027.842105263158|183.05263157894737|
|       Philippines|1018.3434343434343| 98.95454545454545|
|           Germany| 887.0948905109489| 77.97810218978103|
|           England| 864.7222222222222| 82.7555555555555

In [5]:
# 2 Oblicz jaki procent kobiet zarabia ponad 50K w podziale na wykształcenie - wynik zaokrąglij do 2 miejsc po przecinku
df.filter(df.sex == "Female").withColumn("earnings", f.when(f.col("earnings") == ">50K", 1).otherwise(0))\
.groupBy(df.education).agg(f.round(f.avg("earnings"),2).alias("percent")).orderBy(f.col("percent"), ascending = False).show()

+------------+-------+
|   education|percent|
+------------+-------+
|   Doctorate|   0.58|
| Prof-school|   0.46|
|     Masters|   0.33|
|   Bachelors|   0.21|
|  Assoc-acdm|   0.13|
|   Assoc-voc|   0.13|
|     HS-grad|   0.07|
|Some-college|   0.07|
|         9th|   0.03|
|        12th|   0.03|
|     5th-6th|   0.02|
|        11th|   0.02|
|        10th|   0.01|
|     7th-8th|   0.01|
|     1st-4th|    0.0|
|   Preschool|    0.0|
+------------+-------+



In [6]:
# 3 Oblicz ile godzin średnio w ciągu tygodnia pracują osoby Never-married w stosunku do reszty społeczeństwa
df.groupBy("marital-status").agg(f.avg("hours-per-week")).show()

+--------------------+-------------------+
|      marital-status|avg(hours-per-week)|
+--------------------+-------------------+
|           Separated|  39.30146341463415|
|       Never-married|   36.9399981278667|
|Married-spouse-ab...|  39.66985645933014|
|            Divorced| 41.246680171055594|
|             Widowed|  32.97985901309164|
|   Married-AF-spouse| 41.130434782608695|
|  Married-civ-spouse|  43.28485576923077|
+--------------------+-------------------+



In [7]:
# 3.1
df.withColumn("Never-married-status", f.when(f.col("marital-status") == "Never-married", "Never-married").otherwise("rest"))\
.groupBy("Never-married-status").agg(f.avg("hours-per-week")).show()

+--------------------+-------------------+
|Never-married-status|avg(hours-per-week)|
+--------------------+-------------------+
|       Never-married|   36.9399981278667|
|                rest|  42.14526007861779|
+--------------------+-------------------+



In [8]:
# 4 W jakim wieku najczęściej zarabia sie powyżej 50K?
df.select("age", f.when(f.col("earnings") == ">50K",1).otherwise(0).alias("earnings")).\
groupBy("age").agg(f.round(f.avg("earnings"),2).alias("earnings-percent"))\
.orderBy("earnings-percent", ascending=False).show(1)

+---+----------------+
|age|earnings-percent|
+---+----------------+
| 50|            0.43|
+---+----------------+
only showing top 1 row



In [9]:
# 5 W podziale ze względu na zmienną race oblicz stosunek liczby kobiet do mężczyzn
df.select("race", f.when(f.col("sex") == "Male", 1).otherwise(0).alias("M"), 
          f.when(f.col("sex") == "Female", 1).otherwise(0).alias("F"))\
.groupBy("race").agg((f.sum("F") / f.sum("M")).alias("ratio")).show()

+------------------+-------------------+
|              race|              ratio|
+------------------+-------------------+
|             Other| 0.6728395061728395|
|Amer-Indian-Eskimo| 0.6197916666666666|
|             White|0.45071450923125067|
|Asian-Pac-Islander|0.49927849927849927|
|             Black| 0.9910771191841937|
+------------------+-------------------+



In [10]:
# 5.1
df.select("race", f.when(f.col("sex") == "Male", 1).otherwise(0).alias("M"), 
          f.when(f.col("sex") == "Female", 1).otherwise(0).alias("F"))\
.groupBy("race").agg(f.round(f.avg("F"),2).alias("F-percent"), 
                     f.round(f.avg("M"),2).alias("M-percent")).show()

+------------------+---------+---------+
|              race|F-percent|M-percent|
+------------------+---------+---------+
|             Other|      0.4|      0.6|
|Amer-Indian-Eskimo|     0.38|     0.62|
|             White|     0.31|     0.69|
|Asian-Pac-Islander|     0.33|     0.67|
|             Black|      0.5|      0.5|
+------------------+---------+---------+



In [11]:
# 6 Oblicz różnicę pomiędzy średnim czasem pracy osób uzyskujących capital-gain a resztą populacji
df.select("hours-per-week", f.when(f.col("capital-gain") !=0, "with-gain")\
          .otherwise("no-gain").alias("capital-gain")).groupBy(f.col("capital-gain"))\
.agg(f.avg("hours-per-week")).show(5)

+------------+-------------------+
|capital-gain|avg(hours-per-week)|
+------------+-------------------+
|   with-gain|  43.51032448377581|
|     no-gain| 40.158263258400616|
+------------+-------------------+



In [14]:
# 7 Oblicz róznicę wieku do średniego wieku osób z danym poziomem wykształcenia
wndw = Window.partitionBy("education")

df.select("education", "age", f.avg("age").over(wndw).alias("avg-age"),\
          (f.col("age") - f.avg("age").over(wndw)).alias("age-difference")).orderBy("education").show(10)

+---------+---+-----------------+-------------------+
|education|age|          avg-age|     age-difference|
+---------+---+-----------------+-------------------+
|     10th| 58|37.42979635584137|  20.57020364415863|
|     10th| 55|37.42979635584137|  17.57020364415863|
|     10th| 27|37.42979635584137|-10.429796355841368|
|     10th| 25|37.42979635584137|-12.429796355841368|
|     10th| 59|37.42979635584137|  21.57020364415863|
|     10th| 36|37.42979635584137|-1.4297963558413684|
|     10th| 33|37.42979635584137| -4.429796355841368|
|     10th| 67|37.42979635584137|  29.57020364415863|
|     10th| 17|37.42979635584137| -20.42979635584137|
|     10th| 60|37.42979635584137|  22.57020364415863|
+---------+---+-----------------+-------------------+
only showing top 10 rows

